In [27]:
#web_scrapping 
from bs4 import BeautifulSoup
import requests
import pandas as pd





In [6]:
url = 'https://swnsdigital.com/uk/2023/02/these-are-the-top-20-unwanted-fruit-and-vegetables-from-bagged-salad-leaves-to-bananas/'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")


# Find all <li> elements
list_items = soup.find_all('li')


# Initialize an empty list to store data
data = []

# Iterate through each <li> element
for li in list_items:
    # Find <span> elements within the <li> element
    span_elements = li.find_all('span')
    # Extract the text content of each <span> element
    span_texts = [span.text for span in span_elements]
    # Add the extracted data to the list
    data.append(span_texts)

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)  # Adjust column names as needed

# Print the DataFrame
df = df.dropna().reset_index(drop=True)
df= df.rename(columns={0:'top_fruit_veg_wasted'})
df.to_csv('../source/top_fruit_veg_wasted.csv',index=False)


In [7]:

df



,top_fruit_veg_wasted
0,Salad leaves (bagged)
1,Bananas
2,Lettuce (whole)
3,Cucumber
4,Tomatoes
5,Carrots
6,Mushrooms
7,Potato
8,Grapes
9,Strawberries


In [24]:
# Send a GET request to the URL
url = 'https://en.wikipedia.org/wiki/List_of_national_fruits'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing the data
table = soup.find('table', {'class': 'wikitable'})

# Extract data from the table
rows = table.find_all('tr')
data = []
for row in rows:
    columns = row.find_all('td')
    row_data = []
    for column in columns:
        row_data.append(column.get_text().strip())  # Strip whitespace
    if row_data:
        data.append(row_data)

# # Create DataFrame
custom_headers = ['country', 'common_name','scientific_name','image','ref','notes']

# Create DataFrame
df = pd.DataFrame(data[1:], columns=custom_headers)
# Display DataFrame
df.columns
df = df.drop(columns=[ 'scientific_name', 'image', 'ref', 'notes'])

# df.to_csv('../source/national_fruit.csv',index=False)


In [38]:

# Send a GET request to the Wikipedia page
url = "https://en.wikipedia.org/wiki/United_Nations_geoscheme_for_Europe"
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Function to scrape countries for a given region
def scrape_countries(region_name):
    # Find the heading for the specified region
    region_heading = soup.find('span', {'id': region_name})

    # Find the div containing the list of countries for the specified region
    region_div = region_heading.find_next('div', class_='div-col')

    # Initialize a list to store countries
    countries = []

    # Loop through each list item in the div
    for item in region_div.find_all('li'):
        # Extract country name
        country_name = item.text.split('\xa0')[1]

        # Append country name and region to the list
        countries.append((country_name, region_name.replace("_", " ")))

    return countries

# Scrape countries for each region
regions = ['Eastern_Europe', 'Northern_Europe', 'Southern_Europe', 'Western_Europe']
combined_countries = []
for region in regions:
    combined_countries.extend(scrape_countries(region))

# Create DataFrame
df = pd.DataFrame(combined_countries, columns=['Country', 'Region'])

# Display DataFrame
df

df.to_csv('../source/country_region.csv',index=False)